In [ ]:
# default_exp layers

# Layers

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F

from isic.utils import reduce_loss, NoneReduce

In [ ]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)

    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [ ]:
#export
class MixLoss(nn.Module):
    def __init__(self, old_lf, mixup_cb):
        super().__init__()
        self.old_lf = old_lf
        self.mixup_cb = mixup_cb

    def forward(self, pred, yb):
        if self.mixup_cb.pl_module.testing: return self.old_lf(pred, yb)
        with NoneReduce(self.old_lf) as lf:
            self.mixup_cb.yb_1 = self.mixup_cb.yb_1.to(pred.device)
            self.mixup_cb.lam = self.mixup_cb.lam.to(pred.device)
            loss = torch.lerp(lf(pred, self.mixup_cb.yb_1), lf(pred,yb), self.mixup_cb.lam)
        return reduce_loss(loss, getattr(self.old_lf, 'reduction', 'mean'))

In [ ]:
#export
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, eps:float=0.1, reduction='mean'):
        super().__init__()
        self.eps, self.reduction = eps, reduction
    
    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return torch.lerp(nll, loss/c, self.eps)

In [ ]:
from nbdev.export import *
notebook2script('layers.ipynb')

Converted layers.ipynb.
